# Auto-insurance-fall-2017   -   Lamya Moutiq

## Introduction 

### a ) Library importation 

In [ ]:
#pip install pandas==0.25

In [ ]:
#pip install pandas_profiling

In [5]:
import pandas as pd 
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import lightgbm as lgb

### b ) Data importation 

In [6]:
df_train = pd.read_csv('train_auto.csv')
df_test = pd.read_csv('test_auto.csv')

In [7]:
df_train.head()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,SEX,EDUCATION,JOB,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,1,0,0.0,0,60.0,0,11.0,"$67,349",No,$0,z_No,M,PhD,Professional,14,Private,"$14,230",11,Minivan,yes,"$4,461",2,No,3,18.0,Highly Urban/ Urban
1,2,0,0.0,0,43.0,0,11.0,"$91,449",No,"$257,252",z_No,M,z_High School,z_Blue Collar,22,Commercial,"$14,940",1,Minivan,yes,$0,0,No,0,1.0,Highly Urban/ Urban
2,4,0,0.0,0,35.0,1,10.0,"$16,039",No,"$124,191",Yes,z_F,z_High School,Clerical,5,Private,"$4,010",4,z_SUV,no,"$38,690",2,No,3,10.0,Highly Urban/ Urban
3,5,0,0.0,0,51.0,0,14.0,NaN,No,"$306,251",Yes,M,<High School,z_Blue Collar,32,Private,"$15,440",7,Minivan,yes,$0,0,No,0,6.0,Highly Urban/ Urban
4,6,0,0.0,0,50.0,0,NaN,"$114,986",No,"$243,925",Yes,z_F,PhD,Doctor,36,Private,"$18,000",1,z_SUV,no,"$19,217",2,Yes,3,17.0,Highly Urban/ Urban


## Data discovery 

### a ) Pandas Profiling 

In [8]:
profile = ProfileReport(df_train)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [9]:
profile

Number of variables,26
Number of observations,8161
Total Missing (%),1.1%
Total size in memory,1.6 MiB
Average record size in memory,208.0 B
Numeric,11
Categorical,14
Boolean,1
Date,0
Text (Unique),0
Rejected,0


### b ) Insights 

Dataset: 

*   Number of variable : 26 (including 14 categorical features)
*   Number of raw : 8161 
*   Target feature : TARGET_FLAG (request for indemnification of the insured person yes/no) 

Warnings : 


*   Missing values : JOB, YOJ, CAR_AGE, INCOME, HOME_VAL 
*   Zeros : Kidstriv (88%)
*   High cardinality : INCOME, home_val, bluebook, oldclaim
*   High correlation : target_flag/target_amount , homekids/kidsdriv , homekids/age, mvr_pts/clm_freq 

Decisions : 

*   As our target feature is TARGET_FLAG, we will not use TARGET_AMT which is sufficient to predict TARGET_FLAG (If TARGET_AMT>0 => TARGET_FLAG=1 else TARGET_FLAG=0) and will provokes data leakage. 
*   As TARGET_FLAG is unbalanced ( 73,6% of 0), I'm going to move towards boosting methods.   
*   As I planned to use Gradient Boosting Trees which proceed to their own feature selection, I will keep as many features as possible for the moment. 
*   Feature to remove : TARGET_AMT, REVOKED
*   Feature to convert into numerical features : INCOME, home_val, bluebook, OLDCLAIM
*   Feature encoding : 
  -    Hot encoding : PARENT1, MSTATUS, SEX, CAR_USE, RED_CAR, URBANICITY 
  -    Label encoding : EDUCATION, JOB, CAR_TYPE
*   I choose to keep missing values as the models I planned to use deals well with them and make me enable to keep as many information as possible.












## Data pre-processing

### Removing some features

In [10]:
X_train = df_train.drop(columns=['TARGET_FLAG'])
Y_train = df_train['TARGET_FLAG']

In [11]:
X_train = X_train.drop(columns=['TARGET_AMT','INDEX'])

### Conversion into numerical features

In [12]:
def convert_to_numerical_col(df):
  column_to_convert = ['INCOME',
                      'HOME_VAL',
                      'BLUEBOOK',
                      'OLDCLAIM',
                      ]
  for i in column_to_convert:
    df[i].loc[~df[i].isna()] = df[i].loc[~df[i].isna()].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',',''))
    df[i] = pd.to_numeric(df[i])
  return df

X_train =  convert_to_numerical_col(X_train)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### Feature encoding 

#### a) One Hot encoding 

In [13]:
def hot_encoding (df_train, df_test=None): 
  to_encode = ['PARENT1',
               'MSTATUS',
               'SEX',
               'CAR_USE',
               'RED_CAR',
               'URBANICITY',
               'REVOKED']
  for i in to_encode: 
    classes = np.unique(df_train[i])
    Encoder = OneHotEncoder(sparse=False).fit(classes.reshape(-1, 1))
    df_train[i] = Encoder.transform(np.array(df_train[i]).reshape(-1, 1))
    if df_test is not None: 
      df_test[i] = Encoder.transform(np.array(df_test[i]).reshape(-1, 1))
  if df_test is not None: 
    return df_train, df_test
  else: 
    return df_train 

X_train= hot_encoding (X_train)
X_train.head()

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,SEX,EDUCATION,JOB,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,0,60.0,0,11.0,67349.0,1.0,0.0,0.0,1.0,PhD,Professional,14,0.0,14230,11,Minivan,0.0,4461,2,1.0,3,18.0,1.0
1,0,43.0,0,11.0,91449.0,1.0,257252.0,0.0,1.0,z_High School,z_Blue Collar,22,1.0,14940,1,Minivan,0.0,0,0,1.0,0,1.0,1.0
2,0,35.0,1,10.0,16039.0,1.0,124191.0,1.0,0.0,z_High School,Clerical,5,0.0,4010,4,z_SUV,1.0,38690,2,1.0,3,10.0,1.0
3,0,51.0,0,14.0,NaN,1.0,306251.0,1.0,1.0,<High School,z_Blue Collar,32,0.0,15440,7,Minivan,0.0,0,0,1.0,0,6.0,1.0
4,0,50.0,0,NaN,114986.0,1.0,243925.0,1.0,0.0,PhD,Doctor,36,0.0,18000,1,z_SUV,1.0,19217,2,0.0,3,17.0,1.0


#### b) Label encoding 

In [14]:
def label_encoding (df_train, df_test=None): 
  to_encode = ['EDUCATION',
               'JOB',
               'CAR_TYPE']
  for i in to_encode: 
    #df_train[i] = df_train[i].replace(np.nan, 'none', regex=True)
    Encoder = LabelEncoder()
    df_train[i].loc[~df_train[i].isna()] = Encoder.fit_transform(df_train[i].loc[~df_train[i].isna()])
    if df_test is not None: 
      df_test[i].loc[~df_test[i].isna()] = Encoder.transform(df_test[i].loc[~df_test[i].isna()])
  if df_test is not None: 
    return df_train, df_test
  else: 
    return df_train 

X_train= label_encoding (X_train)
X_train.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,SEX,EDUCATION,JOB,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,0,60.0,0,11.0,67349.0,1.0,0.0,0.0,1.0,3,5,14,0.0,14230,11,0,0.0,4461,2,1.0,3,18.0,1.0
1,0,43.0,0,11.0,91449.0,1.0,257252.0,0.0,1.0,4,7,22,1.0,14940,1,0,0.0,0,0,1.0,0,1.0,1.0
2,0,35.0,1,10.0,16039.0,1.0,124191.0,1.0,0.0,4,0,5,0.0,4010,4,5,1.0,38690,2,1.0,3,10.0,1.0
3,0,51.0,0,14.0,NaN,1.0,306251.0,1.0,1.0,0,7,32,0.0,15440,7,0,0.0,0,0,1.0,0,6.0,1.0
4,0,50.0,0,NaN,114986.0,1.0,243925.0,1.0,0.0,3,1,36,0.0,18000,1,5,1.0,19217,2,0.0,3,17.0,1.0


### Standardization 

In [15]:
def standardization(df_train, df_test=None): 
  cols=np.array(df_train.columns)
  scaler = StandardScaler()
  df_train = scaler.fit_transform(df_train)
  if df_test is not None: 
    df_test = scaler.transform(df_test)
    return pd.DataFrame(df_train,columns=cols), pd.DataFrame(df_test,columns=cols)
  return pd.DataFrame(df_train,columns=cols)

X_train = standardization(X_train)
X_train.head()

,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,SEX,EDUCATION,JOB,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,-0.334421,1.763021,-0.646120,0.122358,0.114588,0.389914,-1.199449,-1.223932,1.074976,0.629392,0.401821,-1.224950,-0.768257,-0.175776,1.362319,-1.287232,-1.559447,0.048302,1.037176,0.373691,0.607596,1.696675,0.507036
1,-0.334421,-0.207523,-0.646120,0.122358,0.621214,0.389914,0.792971,-1.223932,1.074976,1.321544,1.213651,-0.722038,1.301648,-0.091445,-1.049422,-1.287232,-1.559447,-0.459982,-0.689370,0.373691,-0.789715,-1.285587,0.507036
2,-0.334421,-1.134838,0.249732,-0.122009,-0.964042,0.389914,-0.237588,0.817039,-0.930253,1.321544,-1.627753,-1.790726,-0.768257,-1.389666,-0.325900,1.256991,0.641253,3.948331,1.037176,0.373691,0.607596,0.293258,0.507036
3,-0.334421,0.719792,-0.646120,0.855458,NaN,0.389914,1.172468,0.817039,1.074976,-1.447066,1.213651,-0.093399,-0.768257,-0.032058,0.397623,-1.287232,-1.559447,-0.459982,-0.689370,0.373691,-0.789715,-0.408451,0.507036
4,-0.334421,0.603877,-0.646120,NaN,1.116005,0.389914,0.689753,0.817039,-0.930253,0.629392,-1.221838,0.158057,-0.768257,0.272009,-1.049422,1.256991,0.641253,1.729590,1.037176,-2.676004,0.607596,1.521248,0.507036


### SUMMARY

In [16]:
X_train = df_train.drop(columns=['TARGET_FLAG'])
Y_train = df_train['TARGET_FLAG']

X_test = df_test.drop(columns=['TARGET_FLAG'])
Y_test = df_test['TARGET_FLAG']

def data_processing(df_train, df_test): 
  
  # Removing some features
  df_train = df_train.drop(columns=['TARGET_AMT','INDEX'])
  df_test = df_test.drop(columns=['TARGET_AMT','INDEX'])

  # Converting into numerical data
  df_train = convert_to_numerical_col(df_train)
  df_test = convert_to_numerical_col(df_test)

  #One hot encoding
  df_train, df_test = hot_encoding (df_train, df_test)

  #Label encoding
  df_train, df_test = label_encoding (df_train, df_test)

  #Standardization 
  df_train, df_test = standardization(df_train, df_test)

  return df_train, df_test

X_train, X_test = data_processing(X_train, X_test)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Modelisation 

In [17]:
# As the logistic regression and XGBoostClassifier do not support Nan values, let's remove raws containing zeros
df_train_without_Nan = df_train.dropna()

X_train_without_Nan= df_train_without_Nan.drop(columns=['TARGET_FLAG'])
Y_train_without_Nan = df_train_without_Nan['TARGET_FLAG']

X_test = df_test.drop(columns=['TARGET_FLAG'])
Y_test = df_test['TARGET_FLAG']

X_train_without_Nan, X_test = data_processing(X_train_without_Nan, X_test)

x_lg_train, x_lg_test, y_lg_train, y_lg_test = train_test_split(X_train_without_Nan, Y_train_without_Nan, test_size=0.3, random_state=42)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### BASELINE 

In [18]:
clf = LogisticRegression(random_state=0).fit(x_lg_train, y_lg_train)
print(clf.score(x_lg_train, y_lg_train))
print(clf.score(x_lg_test, y_lg_test))

0.7960293074923186
0.7695700110253583


In [19]:
clf_pred_train = clf.predict(x_lg_train)
clf_pred_test = clf.predict(x_lg_test)
print(f1_score(y_lg_train, clf_pred_train))
print(f1_score(y_lg_test, clf_pred_test))

0.4950263311878291
0.4877450980392156


### XGBOOST

In [20]:
xgbc = XGBClassifier()
xgbc.fit(x_lg_train, y_lg_train)

print(xgbc.score(x_lg_train, y_lg_train))
print(xgbc.score(x_lg_test, y_lg_test))

0.8352635310801229
0.7772877618522602


In [21]:
xgbc_pred_train = xgbc.predict(x_lg_train)
xgbc_pred_test = xgbc.predict(x_lg_test)
print(f1_score(y_lg_train, xgbc_pred_train))
print(f1_score(y_lg_test, xgbc_pred_test))

0.5996553704767376
0.5036855036855037


### LIGHTGBM

In [22]:
X_train = df_train.drop(columns=['TARGET_FLAG'])
Y_train = df_train['TARGET_FLAG']

X_test = df_test.drop(columns=['TARGET_FLAG'])
Y_test = df_test['TARGET_FLAG']

X_train_with_Nan= df_train.drop(columns=['TARGET_FLAG'])
Y_train_with_Nan = df_train['TARGET_FLAG']

X_train_with_Nan, X_test = data_processing(X_train_with_Nan, X_test)

x_lgm_train, x_lgm_test, y_lgm_train, y_lgm_test = train_test_split(X_train_with_Nan, Y_train_with_Nan, test_size=0.3, random_state=42)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
cat_cols = ['PARENT1',
            'MSTATUS',
            'SEX',
            'CAR_USE',
            'RED_CAR',
            'URBANICITY',
            'REVOKED',
            'EDUCATION',
            'JOB',
            'CAR_TYPE']

cont_cols = ['KIDSDRIV', 
              'AGE', 
              'HOMEKIDS',
              'YOJ', 
              'INCOME', 
              'HOME_VAL',
              'TRAVTIME', 
              'BLUEBOOK', 
              'TIF',
              'OLDCLAIM', 
              'CLM_FREQ',
              'MVR_PTS', 
              'CAR_AGE']

label_col = 'TARGET_FLAG'

config = {
"fit_params": {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    "is_unbalanced" : 'true',
    "num_threads": 40,
    "eval_at": 10,
    "max_depth": 16,
    "num_leaves": 100,
},
"num_boost_round": 150,
"early_stopping_rounds": 10,
"verbose_eval": 50,
}

In [24]:
lgb_training_dataset = lgb.Dataset(
        data=x_lgm_train, 
        label=y_lgm_train,
        categorical_feature=cat_cols,
        free_raw_data=False,
    )

lgb_test_dataset = lgb.Dataset(
        data=x_lgm_test,
        label=y_lgm_test,
        categorical_feature=cat_cols,
        free_raw_data=False,
    )

In [25]:
train_params = dict(
        params=config["fit_params"],
        train_set=lgb_training_dataset,
        valid_sets=[lgb_test_dataset],
        valid_names=["valid"],
        num_boost_round=config["num_boost_round"],
        early_stopping_rounds=config["early_stopping_rounds"],
        categorical_feature=cat_cols,
        verbose_eval=config["verbose_eval"],
    )

gbm = lgb.train(**train_params)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[30]	valid's binary_logloss: 0.447398


In [26]:
y_pred_train = gbm.predict(lgb_training_dataset.get_data())
y_pred_test = gbm.predict(lgb_test_dataset.get_data())

In [27]:
#rounding the values
y_pred_train=y_pred_train.round(0)
#converting from float to integer
y_pred_train=y_pred_train.astype(int)

#rounding the values
y_pred_test=y_pred_test.round(0)
#converting from float to integer
y_pred_test=y_pred_test.astype(int)

In [28]:
print(accuracy_score(y_lgm_train,y_pred_train))
print(accuracy_score(y_lgm_test,y_pred_test))

print(f1_score(y_lgm_train, y_pred_train))
print(f1_score(y_lgm_test, y_pred_test))

0.9275210084033614
0.7913434054716211
0.8466666666666667
0.5081809432146295


## HYPEROPT

In [ ]:
from hyperopt import hp, tpe, Trials, STATUS_OK, STATUS_RUNNING
from hyperopt.fmin import fmin

def objective(params):
  train_params = dict(
        params=params,
        train_set=lgb_training_dataset,
        valid_sets=[lgb_test_dataset],
        valid_names=["valid"],
        num_boost_round=config["num_boost_round"],
        early_stopping_rounds=config["early_stopping_rounds"],
        categorical_feature=cat_cols,
        verbose_eval=config["verbose_eval"],
    )
  model_lgb = lgb.train(**train_params)   
  y_pred_test = model_lgb.predict(lgb_test_dataset.get_data())

  #rounding the values
  y_pred_test=y_pred_test.round(0)
  #converting from float to integer
  y_pred_test=y_pred_test.astype(int)
  
  score = accuracy_score(y_lgm_test,y_pred_test)

  return {
      'loss': score,
      'status': STATUS_OK,
      'stats_running': STATUS_RUNNING
  }


space = {
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "binary_logloss",
        "is_unbalanced" : 'true',
        "num_threads": 40,
        "eval_at": 10,
        'max_depth' : 16,
        'num_leaves': hp.choice('num_leaves', np.arange(80, 110, 10)),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01),np.log(0.1))}

trials= Trials()
best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=50,
                trials=trials,
               verbose= 2)

print(best)

In [46]:
best_param = {
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "binary_logloss",
        "is_unbalanced" : 'true',
        "num_threads": 40,
        "eval_at": 10,
        'max_depth' : 16,
        'num_leaves': 100,
       # 'learning_rate': 0.010089213575724754
       }

## CROSS VALIDATION OF THE FINAL MODEL

In [47]:
from sklearn.model_selection import KFold 

k = 4
kf = KFold(n_splits=k, random_state=None)

models = []
acc_score = []

for train_index , test_index in kf.split(X_train_with_Nan):
    train_x = x_lgm_train.loc[train_index]
    train_y = y_lgm_train.loc[train_index]
    
    test_x = x_lgm_test.loc[test_index]
    test_y = y_lgm_test.loc[test_index]
    
    d_training = lgb.Dataset(train_x, label=train_y,categorical_feature=cat_cols, free_raw_data=False)
    d_test = lgb.Dataset(test_x, label=test_y,categorical_feature=cat_cols, free_raw_data=False)
    
    train_params = dict(
        params=best_param,
        train_set=lgb_training_dataset,
        valid_sets=[lgb_test_dataset],
        valid_names=["valid"],
        num_boost_round=config["num_boost_round"],
        early_stopping_rounds=config["early_stopping_rounds"],
        categorical_feature=cat_cols,
        verbose_eval=config["verbose_eval"],
    )

    model = lgb.train(**train_params)
    models.append(model)

    y_pred_test = model.predict(lgb_test_dataset.get_data())

    #rounding the values
    y_pred_test=y_pred_test.round(0)
    #converting from float to integer
    y_pred_test=y_pred_test.astype(int)
    
    score = accuracy_score(y_lgm_test,y_pred_test)
    acc_score.append(score)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an

Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[30]	valid's binary_logloss: 0.447398
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[30]	valid's binary_logloss: 0.447398
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[30]	valid's binary_logloss: 0.447398
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[30]	valid's binary_logloss: 0.447398
accuracy of each fold - [0.7913434054716211, 0.7913434054716211, 0.7913434054716211, 0.7913434054716211]
Avg accuracy : 0.7913434054716211
